In [1]:
%load_ext nb_mypy

Version 1.0.5


In [2]:
from loguru import logger
from s3path import S3Path
from datetime import datetime

from sm_pipelines_oo.config_loader.implementations import MockConfigLoader
from sm_pipelines_oo.steps.framework_processing_step import StepFactory
from sm_pipelines_oo.shared_config_schema import SharedConfig, Environment
from sm_pipelines_oo.aws_connector.interface import AWSConnectorInterface
from sm_pipelines_oo.aws_connector.concrete_connectors import create_aws_connector

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/thomas-22/.config/sagemaker/config.yaml


In [3]:
_PROJECT_BUCKET = S3Path('/smp-oo-example')
_DATA_PREFIX = 'examples/data/'
output_filename = f'output-{datetime.now()}'

env: Environment = 'dev'
_shared_config_dict={
    'project_name': 'test',
    'project_version': '0',
    'region': 'us-east-1',
    'project_bucket_name': _PROJECT_BUCKET.bucket,
    'role_name': 'sagemaker_pipelines_role'
}
_fw_processor_config_dict = {
    'step_name': 'preprocessing',
    'step_factory_class': 'FrameworkProcessor',
    'processor_init_config': {
        'framework_version': '0.23-1',
        'estimator_cls_name': 'SKLearn',
        'instance_count': 1,
        'instance_type': 'ml.m5.xlarge',
        'env': {'OUTPUT_FILENAME': output_filename},
    },
    'processor_run_config': {
        'code': 'preprocess.py',
        'source_dir': 'worker_code/preprocess',
        'inputs': {
            'input_1': (_PROJECT_BUCKET / _DATA_PREFIX / "input_1" ).as_uri(),
        },
        'outputs': {
            'output_1': (_PROJECT_BUCKET / _DATA_PREFIX / "output_1").as_uri(),
        },
    },
    'shared_config': _shared_config_dict
}
_shared_config = SharedConfig(**_shared_config_dict)
_aws_connector: AWSConnectorInterface = create_aws_connector(
    shared_config=_shared_config,
    environment=env,
)
factory = StepFactory(
    step_config_dict=_fw_processor_config_dict,
    role_arn=_aws_connector.role_arn,
    sm_session=_aws_connector.sm_session,
    pipeline_session=_aws_connector.pipeline_session,
)

In [4]:
factory.run_processor()

INFO:sagemaker:Creating processing-job with name sklearn-2024-03-15-14-09-41-432


..........................Found existing installation: typing 3.7.4.3
Uninstalling typing-3.7.4.3:
  Successfully uninstalled typing-3.7.4.3
[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: pip install --upgrade pip



In [5]:
from smp_oo_examples.validation import check_output

check_output(
    output_path_s3=_PROJECT_BUCKET / _DATA_PREFIX / "output_1" / output_filename,
)

   a   b   c
0  2   8  14
1  4  10  16
2  6  12  18
Success


In [6]:
print(datetime.now())

2024-03-15 10:14:44.077814
